In [1]:
import pandas as pd
import requests
import json
import time
from datetime import datetime
import os
from settings import FIRST_FILE_INDEX, LAST_FILE_INDEX, TOKEN

In [2]:
first_file_index = FIRST_FILE_INDEX
last_file_index = LAST_FILE_INDEX

file_paths = [os.path.join("data", "tssb_data_3M", f"file-{index}.jsonl") for index in range(first_file_index, last_file_index+1)]

In [3]:
data_list = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        for line in file:
            data_dict = json.loads(line)
            data_list.append(data_dict)

df = pd.DataFrame(data_list)

del data_list, file_paths

In [4]:
# A Preview of TSSB-Data
df.head()

,project,commit_sha,parent_sha,file_path,project_url,likely_bug,comodified,in_function,diff,before,after,sstub_pattern,edit_script
0,ngraph,14b740b1baec641e25142bcdc7b42f1b2e631b34,73b56168da47fdc414d69223367e527c8e96d1a8,ngraph/transformers/passes/hetrpasses.py,https://github.com/rsumner31/ngraph,True,False,True,"@@ -30,7 +30,7 @@ class DeviceAssignPass(Graph...",if len ( op . metadata [ 'device_id' ] ) == 1 ...,if 'device_id' in op . metadata and isinstance...,MORE_SPECIFIC_IF,"[[""Insert"", [""if_statement"", 3, 9, 4, 43], [""b..."
1,ngraph,1445e0684fbcca2ec49a5f1becf1345159b7ba6a,4eb8eed57e506e8a2745b298340666e9d7e5ce58,ngraph/op_graph/op_graph.py,https://github.com/rsumner31/ngraph,True,False,True,"@@ -787,7 +787,7 @@ def set_item(tensor, item,...",raise ValueError ( 'Invalid slice in item {}' ...,raise ValueError ( 'Invalid slice (negative st...,CHANGE_STRING_LITERAL,"[[""Update"", [""string:'Invalid slice in item {}..."
2,ngraph,3136396b434813fc8e6fa5eacb48cbc6ed04506a,a8524402bb2c1fd9c63f4af29ee01147f5360c42,ngraph/op_graph/op_graph.py,https://github.com/rsumner31/ngraph,True,False,True,"@@ -1654,7 +1654,7 @@ class RoleCastOp(AxesCas...","def copy_op_with_new_args ( self , args ) : re...","def copy_with_new_args ( self , args ) : retur...",SINGLE_TOKEN,"[[""Update"", [""identifier:copy_op_with_new_args..."
3,ngraph,85e84fa359d23357cb1d59cff1f6da5ff671e5ec,5e55a9cb79f1ca70521ee4a37a771647a713cfc2,ngraph/transformers/passes/hetrpasses.py,https://github.com/rsumner31/ngraph,True,False,True,"@@ -31,7 +31,7 @@ class DeviceAssignPass(Graph...",op . metadata [ 'device_id' ] = '1',op . metadata [ 'device_id' ] = op . metadata ...,SINGLE_STMT,"[[""Insert"", [""assignment"", 3, 13, 3, 43], [""su..."
4,ngraph,41c47b854c352fec23a086918436227e15f53a22,0e7c49699718335f045458c73c0da25082aa4d3b,ngraph/frontends/tensorflow/tests/test_example...,https://github.com/rsumner31/ngraph,True,False,True,"@@ -89,7 +89,7 @@ class TestExamples(ImporterT...",if self . transformer_name == 'hetr' : pytest ...,"if self . transformer_name in [ 'hetr' , 'gpu'...",SINGLE_STMT,"[[""Insert"", [""comparison_operator"", 3, 12, 3, ..."


In [7]:
# Enter your personal token.
token = TOKEN

# The limit of GitHub API requests per hour.
github_limit = 5000

dataset_size = df.shape[0]
numberof_partitions = dataset_size // github_limit + 1
partitions_boundaries = [github_limit * (i) for i in range(numberof_partitions)] 
partitions_boundaries += [partitions_boundaries[-1] + dataset_size % github_limit]

#Make it with os
extracted_data_path = os.path.join("data", "commits")

for baundary_index in range(numberof_partitions):
    commits = []
    errors = []
    
    for data_index in range(partitions_boundaries[baundary_index], partitions_boundaries[baundary_index + 1]):
        username, reponame = df["project_url"][data_index].split('/')[-2:]
        commit_sha = df["commit_sha"][data_index]

        headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/vnd.github.v3+json"
        }

        commit_url = f"https://api.github.com/repos/{username}/{reponame}/commits/{commit_sha}"
        commit_response = requests.get(commit_url, headers=headers)

        if commit_response.status_code == 200:
            commit_details = commit_response.json()
            commits.append({
                "username": username,
                "reponame": reponame,
                "commit_sha": commit_sha,
                "message": commit_details["commit"]["message"],
                "diff": df["diff"][data_index],
                "sstub_pattern": df["sstub_pattern"][data_index],
                "likely_bug": str(df["likely_bug"][data_index])
            })

        else:
            print("Error:", commit_response.status_code, commit_response.text, username, reponame, commit_sha)

            errors.append({
                "username": username,
                "reponame": reponame,
                "commit_sha": commit_sha,
                "response_code": commit_response.status_code,
                "response_text": commit_response.text
            })
    
    with open(os.path.join(extracted_data_path, ("commits_" + str(baundary_index) + ".json")), "w") as file:
        json.dump(commits, file)
    
    with open(os.path.join(extracted_data_path, ("errors_" + str(baundary_index) + ".json")), "w") as file:
        json.dump(errors, file)

    print("Last Index Observed: " + str(baundary_index) + " | Time: " + str(datetime.now()))

    time.sleep(3600)

del partitions_boundaries, commits, errors

Error: 401 {"message":"Bad credentials","documentation_url":"https://docs.github.com/rest"} rsumner31 ngraph 14b740b1baec641e25142bcdc7b42f1b2e631b34
Error: 401 {"message":"Bad credentials","documentation_url":"https://docs.github.com/rest"} rsumner31 ngraph 1445e0684fbcca2ec49a5f1becf1345159b7ba6a
Error: 401 {"message":"Bad credentials","documentation_url":"https://docs.github.com/rest"} rsumner31 ngraph 3136396b434813fc8e6fa5eacb48cbc6ed04506a
Error: 401 {"message":"Bad credentials","documentation_url":"https://docs.github.com/rest"} rsumner31 ngraph 85e84fa359d23357cb1d59cff1f6da5ff671e5ec
Error: 401 {"message":"Bad credentials","documentation_url":"https://docs.github.com/rest"} rsumner31 ngraph 41c47b854c352fec23a086918436227e15f53a22
Error: 401 {"message":"Bad credentials","documentation_url":"https://docs.github.com/rest"} rsumner31 ngraph 7d39d25f8ba809394121a918bcd0e8706f4720ea
Error: 401 {"message":"Bad credentials","documentation_url":"https://docs.github.com/rest"} rsumner3

KeyboardInterrupt: 